In [2]:
#Importing functions
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly as px

import scipy.stats as scs
from scipy.stats import t
from scipy.stats import norm
import numpy as np

import statsmodels.api as sm
from statsmodels.formula.api import ols
from Functions import func_rename, func_conversion


Attribute Information:

    mpg: continuous
    cylinders: multi-valued discrete
    displacement: continuous
    horsepower: continuous
    weight: continuous
    acceleration: continuous
    model year: multi-valued discrete
    origin: multi-valued discrete
    car name: string (unique for each instance)

Missing Attribute Values: horsepower has 6 missing values

In [4]:
# Importing and cleaning data
filepath = "../Lab/Data/cars.csv" # Creating filepath
df = pd.read_csv(filepath) # reading csv with pandas

# Adding columns
df[["name", "test"]] = df["name"].str.split(" ", 1, expand=True)  # Creating a new column
df.rename(columns={"name": "make", "test": "model", "mpg": "lper100km"}, inplace=True) # Renaming columns
cars = df # Creating a new dataframe

# Handling spelling mistakes:
#   The spelling mistakes were found by manually looking through the data.
#   func_rename can be found in the Functions.py file 
func_rename(cars,"vw","volkswagen"), func_rename(cars,"vokswagen","volkswagen") 
func_rename(cars,"chevy","chevrolet"), func_rename(cars,"chevroelt","chevrolet")
func_rename(cars,"maxda","mazda"), func_rename(cars,"toyouta", "toyota")
func_rename(cars,"mercedes-benz", "mercedes"),

# Converting the weight and miles per gallon into the metric system.
#  func_conversion can be found in the Functions.py file
func_conversion(cars, "weight")
func_conversion(cars, "lper100km")

#dropping nan due to horsepower is missing 6 values
cars.dropna(inplace=True)

C:\Users\Agam\AppData\Local\Temp\ipykernel_9296\156510797.py:6: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[["name", "test"]] = df["name"].str.split(" ", 1, expand=True)  # Creating a new column
c:\Users\Agam\Documents\Github\Statistical_Analysis_Alexander_Andersson\Lab\Functions.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[column][p] = round(dataframe[column][p]*0.45359237) #Correct math for converting
c:\Users\Agam\Documents\Github\Statistical_Analysis_Alexander_Andersson\Lab\Functions.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

## Auto-mpg dataset statistical analysis
The point of this analysis is to check for how 